In [1]:
import time
from absl import app, flags, logging
from absl.flags import FLAGS
import cv2
import numpy as np

import tensorflow.compat.v1 as tf
tf.app.flags.DEFINE_string('f', '', 'kernel')
from yolov3_tf2.models import (
    YoloV3, YoloV3Tiny
)
from yolov3_tf2.dataset import transform_images, load_tfrecord_dataset
from yolov3_tf2.utils import draw_outputs



flags.DEFINE_string('classes', './data/coco.names', 'path to classes file')
flags.DEFINE_string('weights', './checkpoints/yolov3.tf',
                    'path to weights file')
flags.DEFINE_boolean('tiny', False, 'yolov3 or yolov3-tiny')
flags.DEFINE_integer('size', 416, 'resize images to')
flags.DEFINE_integer('batch_size', 1, 'batch size')
flags.DEFINE_string('image', './data/people.jpg', 'path to input image')
flags.DEFINE_string('tfrecord', None, 'tfrecord instead of image')
flags.DEFINE_string('output', './output.jpg', 'path to output image')
flags.DEFINE_integer('num_classes', 80, 'number of classes in the model')


def main(_argv):
    physical_devices = tf.config.experimental.list_physical_devices('GPU')
    if len(physical_devices) > 0:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)

    if FLAGS.tiny:
        yolo = YoloV3Tiny(classes=FLAGS.num_classes)
    else:
        yolo = YoloV3(classes=FLAGS.num_classes)

    yolo.load_weights(FLAGS.weights).expect_partial()
    logging.info('weights loaded')

    class_names = [c.strip() for c in open(FLAGS.classes).readlines()]
    logging.info('classes loaded')

    if FLAGS.tfrecord:
        dataset = load_tfrecord_dataset(
            FLAGS.tfrecord, FLAGS.classes, FLAGS.size)
        dataset = dataset.shuffle(512)
        img_raw, _label = next(iter(dataset.take(1)))
    else:
        img_raw = tf.image.decode_image(
            open(FLAGS.image, 'rb').read(), channels=3)

    img = tf.expand_dims(img_raw, 0)
    img = transform_images(img, FLAGS.size)

    t1 = time.time()
    boxes, scores, classes, nums = yolo(img)
    t2 = time.time()
    logging.info('time: {}'.format(t2 - t1))

    logging.info('detections:')
    for i in range(nums[0]):
        logging.info('\t{}, {}, {}'.format(class_names[int(classes[0][i])],
                                           np.array(scores[0][i]),
                                           np.array(boxes[0][i])))

    img = cv2.cvtColor(img_raw.numpy(), cv2.COLOR_RGB2BGR)
    img = draw_outputs(img, (boxes, scores, classes, nums), class_names)
    cv2.imwrite(FLAGS.output, img)
    logging.info('output saved to: {}'.format(FLAGS.output))


if __name__ == '__main__':
    try:
        app.run(main)
    except SystemExit:
        pass


I0102 10:22:56.650821 29592 <ipython-input-1-d3701337714a>:40] weights loaded
I0102 10:22:56.654810 29592 <ipython-input-1-d3701337714a>:43] classes loaded
I0102 10:22:58.691436 29592 <ipython-input-1-d3701337714a>:60] time: 2.0286202430725098
I0102 10:22:58.692386 29592 <ipython-input-1-d3701337714a>:62] detections:
I0102 10:22:58.696403 29592 <ipython-input-1-d3701337714a>:64] 	person, 0.9990279078483582, [0.5679409  0.38697928 0.6977561  0.78918546]
I0102 10:22:58.700366 29592 <ipython-input-1-d3701337714a>:64] 	person, 0.9925042390823364, [0.35881883 0.33818758 0.45891166 0.78770685]
I0102 10:22:58.704358 29592 <ipython-input-1-d3701337714a>:64] 	person, 0.9908876419067383, [0.07741459 0.40896302 0.20195623 0.8045911 ]
I0102 10:22:58.708345 29592 <ipython-input-1-d3701337714a>:64] 	person, 0.9905360341072083, [0.01139769 0.37697214 0.09651744 0.6519876 ]
I0102 10:22:58.711337 29592 <ipython-input-1-d3701337714a>:64] 	person, 0.9901893734931946, [0.7241706  0.35416576 0.8466498  0.7